In [1]:
import pandas as pd
import numpy as np
import sys
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [20]:
cols = 4
col_names = []
for i in range(cols):
    col_names.append( 'r' + str(i+1))

rows = 100000
train_rows = int(rows * 0.75)

df = pd.DataFrame(np.random.randn(rows, cols), columns=col_names)
X = df
y = df.r1/(df.r2 + 0.0000000000000001) + df.r3 * df.r4

X_train = X[:train_rows]
y_train = y[:train_rows]
X_test = X[train_rows:]
y_test = y[train_rows:]


In [21]:
mode = 'regression'
params =  {
	'task': 'train',
	'boosting_type': 'gbdt',
	'objective': 'regression' if mode == 'regression' else 'binary',
	'metric': 'rmse',
	"learning_rate": 0.01,
	"num_leaves": 200,
	"feature_fraction": 0.70,
	"bagging_fraction": 0.70,
	'bagging_freq': 4,
    "verbosity" : -1,
	"reg_alpha": 0.3,
	"reg_lambda": 0.1,
	#"min_split_gain":0.2,
	"min_child_weight":10,
	'zero_as_missing':True,
    'num_threads': 4,
    }

In [22]:
prev_rmse = None
for d in range(2,16):
    #lgb_model = lgb.LGBMModel()
    est = lgb.sklearn.LGBMRegressor(max_depth = d)
    est.set_params( **params )
    est.fit(X_train, y_train)
    predict = est.predict( X_test)
    mse = mean_squared_error(y_test, predict)
    rmse = mse ** 0.5 
    print(rmse, np.mean(y_test))
    if (not prev_rmse is None) and prev_rmse < rmse:
        print('early finish at {} iteration.'.format(d))
        break
    prev_rmse = rmse
    
print('best score: {}'.format(prev_rmse))
    

137.7106575725666 -0.47051017284695573
137.27575457473847 -0.47051017284695573
135.88828113718765 -0.47051017284695573
134.70662125004534 -0.47051017284695573
134.06661579774794 -0.47051017284695573
133.85786274744692 -0.47051017284695573
133.81059946709385 -0.47051017284695573
133.79846605503204 -0.47051017284695573
133.79973731991208 -0.47051017284695573
early finish at 10 iteration.
best score: 133.79846605503204


In [ ]:
#est.get_params()
#params = {'key': [1,2,3] }
#param_name = list(params.keys())[0]
#param_values = params[param_name]
#print(param_values[0])
y_test